In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/src/script.ipynb
/kaggle/lib/kaggle/gcp.py
/kaggle/input/bengalimodel/resnet34_fold0.bin
/kaggle/input/bengalimodel/resnet34_fold3.bin
/kaggle/input/bengalimodel/resnet34_fold2.bin
/kaggle/input/bengalimodel/resnet34_fold1.bin
/kaggle/input/pretrained-models/pretrained-models.pytorch-master/.gitignore
/kaggle/input/pretrained-models/pretrained-models.pytorch-master/README.md
/kaggle/input/pretrained-models/pretrained-models.pytorch-master/setup.cfg
/kaggle/input/pretrained-models/pretrained-models.pytorch-master/LICENSE.txt
/kaggle/input/pretrained-models/pretrained-models.pytorch-master/setup.py
/kaggle/input/pretrained-models/pretrained-models.pytorch-master/requirements.txt
/kaggle/input/pretrained-models/pretrained-models.pytorch-master/.travis.yml
/kaggle/input/pretrained-models/pretrained-models.pytorch-master/pretrainedmodels/__init__.py
/kaggle/input/pretrained-models/pretrained-models.pytorch-master/pretrainedmodels/utils.py
/kaggle/input/pretrained-models/pretrained-m

In [2]:
import sys
pt_models = '/kaggle/input/pretrained-models/pretrained-models.pytorch-master/'
sys.path.insert(0, pt_models)
import pretrainedmodels

In [3]:
import glob
import torch
import albumentations
import pandas as pd
import numpy as np
from tqdm import tqdm
from PIL import Image
import joblib
import torch.nn as nn
from torch.nn import functional as F

In [4]:
TRAIN_BAT_SIZE = 32
TEST_BAT_SIZE = 8
MODEL_MEAN=(0.485, 0.456, 0.406)
MODEL_STD=(0.229, 0.224, 0.225)
IMG_HT = 137
IMG_WD = 236
DEVICE='cuda'

In [5]:
class ResNet34(nn.Module):
    def __init__(self, pretrained):
        super(ResNet34, self).__init__()
        if pretrained:
            self.model = pretrainedmodels.__dict__['resnet34'](pretrained = 'imagenet')
        else:
            self.model = pretrainedmodels.__dict__['resnet34'](pretrained = None)
        self.l0 = nn.Linear(512, 168)
        self.l1 = nn.Linear(512, 11)
        self.l2 = nn.Linear(512, 7)

    def forward(self, X):
        s = X.shape
        X = (X.reshape(1, s[0], s[1], s[2]))
        bs, _, _, _ = X.shape
        X = self.model.features(X)
        X = nn.functional.adaptive_avg_pool2d(X, 1).reshape(bs, -1)
        l0 = self.l0(X)
        l1 = self.l1(X)
        l2 = self.l2(X)
        return l0, l1, l2

In [6]:
class BengaliDatasetTest:
    def __init__(self, df, img_ht, img_wd, mean, std):
        # pdb.set_trace()
        self.image_ids = df.image_id.values
        self.img_arr = df.iloc[:, 1:].values
        
        self.aug = albumentations.Compose([
            albumentations.Resize(img_ht, img_wd, always_apply = True),
            albumentations.Normalize(mean, std, always_apply = True)
        ])

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, item):
        # pdb.set_trace()
        # print(self.image_ids[item])
        image = self.img_arr[item, :]
        img_id = self.image_ids[item]
        image = image.reshape(137, 236).astype(float)
        image = Image.fromarray(image).convert('RGB')
        image = self.aug(image = np.array(image))['image']
        # pdb.set_trace()
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        return {
            'image' : torch.tensor(image, dtype = torch.float),
            'image_id' : img_id
        }

In [7]:
model = ResNet34(pretrained=False)
model.load_state_dict(torch.load('/kaggle/input/bengalimodel/resnet34_fold1.bin'))
model.to(DEVICE)
with torch.no_grad():
    model.eval()

In [8]:
import pdb
predictions = []
for file_idx in range(4):
    df = pd.read_parquet(f'../input/bengaliai-cv19/test_image_data_{file_idx}.parquet')
    dataset = BengaliDatasetTest(df = df, img_ht = IMG_HT, img_wd = IMG_WD, mean = MODEL_MEAN, std = MODEL_STD)
    data_loader = torch.utils.data.DataLoader(dataset, batch_size = TEST_BAT_SIZE,
                                        shuffle = False,
                                        num_workers = 4)
    for bi, d in enumerate(dataset):
        image = d['image']
        img_id = d['image_id']
        image = image.to(DEVICE, dtype = torch.float)
        print(image.shape)
        g, v, c = model(image)
        x = g
#         pdb.set_trace()
        g = int(torch.argmax(g, axis = 1))
        v = int(torch.argmax(v, axis = 1))
        c = int(torch.argmax(c, axis = 1))
#         for i, imid in range(img_id):
        predictions.append((f'{img_id}_grapheme_root', g))
        predictions.append((f'{img_id}_vowel_diacritic', v))
        predictions.append((f'{img_id}_consonant_diacritic', c))

torch.Size([3, 137, 236])
torch.Size([3, 137, 236])
torch.Size([3, 137, 236])
torch.Size([3, 137, 236])
torch.Size([3, 137, 236])
torch.Size([3, 137, 236])
torch.Size([3, 137, 236])
torch.Size([3, 137, 236])
torch.Size([3, 137, 236])
torch.Size([3, 137, 236])
torch.Size([3, 137, 236])
torch.Size([3, 137, 236])


## END